In [22]:
from gurobipy import Model, GRB
import time
import pandas as pd
import numpy as np
from pylinerating.ieee738 import thermal_rating
from pylinerating import conductor
import matplotlib.pyplot as plt
from tqdm import tqdm
plt.rcParams["figure.figsize"] = (20,8)
import pickle
import json
from IPython.display import Image
from tqdm import tqdm

In [23]:
# unserved_energy = {}
# for capacity in BESS_capacities: #capacity in GWmin
#     unserved_energy[capacity] = [] #unserved energy in GWmin
        
# duration_between_events_with_unserved_energy = {}
# for SOC_opt in BESS_capacities: #capacity in GWmin
#     duration_between_events_with_unserved_energy[SOC_opt] = {}
#     for PR_opt in PR_opts:
#         duration_between_events_with_unserved_energy[SOC_opt][PR_opt] = [] #duration between events in min

In [24]:
with open('unserved_energy_events.pkl', 'rb') as outfile:
    unserved_energy = pickle.load(outfile)

In [25]:
with open('duration_between_events_with_unserved_energy.pkl', 'rb') as outfile:
    duration_between_events_with_unserved_energy = pickle.load(outfile)

In [26]:
np.array(list(unserved_energy.keys())) * 1000/60

In [27]:
num_years_to_plot = 3
for capacity in list(unserved_energy.keys())[0:4]:
    #print(str(capacity * 1000/60) + ' MWh')
    x = []
    y = []
    year_end_count = 0
    for item in unserved_energy[capacity]:
        if item != 'year_end' and year_end_count < num_years_to_plot:
            #print(item)
            x.append(list(item.keys())[0])
            y.append(list(item.values())[0] * 1000/60)
        else:
            year_end_count += 1
        
    plt.plot(x, y, 'o', markersize=10, label = str(capacity * 1000/60) + ' MWh BESS')
    plt.xlabel('Duration of Unserved Energy Event [min]', fontsize = 20)
    plt.ylabel('Unserved Energy [MWh]', fontsize = 20)
    plt.title('Unserved Energy [MWh] vs Duration of Unserved Energy for Installed BESS Capacities [MWh]', fontsize = 20)
    plt.legend()

In [28]:
# Total unserved energy per capacity
num_years_to_plot = 15
num_caps_to_plot = 100
caps = []
ues = []
for capacity in list(unserved_energy.keys())[0:num_caps_to_plot]:
    caps.append(capacity * 1000/60)
    #print(str(capacity * 1000/60) + ' MWh')
    ue = []
    year_end_count = 0
    for item in unserved_energy[capacity]:
        if item != 'year_end' and year_end_count < num_years_to_plot:
            #print(item)
            ue.append(list(item.values())[0] * 1000/60)
        else:
            year_end_count += 1
    ues.append(sum(ue))
        
plt.plot(caps, ues)
plt.xlabel('Installed BESS Capacity [MWh]', fontsize = 20)
plt.ylabel('Total Unserved Energy [MWh]', fontsize = 20)
plt.title('Total Unserved Energy [MWh] vs Installed BESS Capacity [MWh]', fontsize = 20)
plt.legend()

In [29]:
# Total duration of unserved energy per capacity
# Total unserved energy per capacity
num_years_to_plot = 15
num_caps_to_plot = 100
caps = []
durs = []
for capacity in list(unserved_energy.keys())[0:num_caps_to_plot]:
    caps.append(capacity * 1000/60)
    #print(str(capacity * 1000/60) + ' MWh')
    dur = []
    year_end_count = 0
    for item in unserved_energy[capacity]:
        if item != 'year_end' and year_end_count < num_years_to_plot:
            #print(item)
            dur.append(list(item.keys())[0] * 1000/60)
        else:
            year_end_count += 1
    durs.append(sum(dur))
        
plt.plot(caps, durs)
plt.xlabel('Installed BESS Capacity [MWh]', fontsize = 20)
plt.ylabel('Total Duration of Unserved Energy Events [min]', fontsize = 20)
plt.title('Total Duration of Unserved Energy Events [min] vs Installed BESS Capacity [MWh]', fontsize = 20)
plt.legend()

In [30]:
BESS_capacities_MWh = np.arange(5,25,5) * 60/1000

PR_opt = 1

for capacity in BESS_capacities_MWh:
    y = []
    for item in duration_between_events_with_unserved_energy[capacity][PR_opt]:
        if item != 'year_end':
            y.append(item)
    y = np.array(y)
    x = np.arange(y.shape[0])
    
    plt.scatter(x, y, label = str(capacity * 1000/60) + ' MWh BESS')
    #plt.xlim([0,10000])
    plt.xlabel('Served Energy Events [-]', fontsize = 20)
    plt.ylabel('Duration Between Unserved Energy Events [min]', fontsize = 20)
    plt.title('Dur. Bwn Unserved En. Events [min] vs Served En. Events [-] with DTLR ' 
              + 'for Diff. Caps', fontsize = 25)
    plt.legend(fontsize = 15)
    plt.grid()

In [31]:
BESS_capacities_MWh = np.arange(5,10,5) * 60/1000

PR_opt = 1

for capacity in BESS_capacities_MWh:
    y = []
    for item in duration_between_events_with_unserved_energy[capacity][PR_opt]:
        if item != 'year_end':
            y.append(item)
    y = np.array(y)
    x = np.arange(y.shape[0])
    
    plt.scatter(x, y, label = str(capacity * 1000/60) + ' MWh BESS')
    plt.xlim([0,100])
    plt.xlabel('Unserved Energy Events [-]', fontsize = 20)
    plt.ylabel('Duration Between Unserved Energy Events [min]', fontsize = 20)
    plt.title('Dur. Bwn Unserved En. Events [min] vs Served En. Events [-] with DTLR ' 
              + 'for Diff. Caps', fontsize = 25)
    plt.legend(fontsize = 15)
    plt.grid()